In [1]:
import zipfile

local_zip = './datasets/horse-or-human.zip'
data_dir = './datasets/horse-or-human'

validation_local_zip = './datasets/horse-or-human-validation.zip'
validation_data_dir = './datasets/horse-or-human-validation'

# Unzip training set
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(data_dir)

# Unzip validation set
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(validation_data_dir)

zip_ref.close()

In [2]:
from os.path import join
from os import listdir

train_horse_dir = join(data_dir, 'horses')
train_human_dir = join(data_dir, 'humans')

validation_horse_dir = join(validation_data_dir, 'horses')
validation_human_dir = join(validation_data_dir, 'humans')

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.nn import relu, sigmoid

model = Sequential(
    [
        Conv2D(16, (3, 3), activation=relu, input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        Conv2D(32, (3, 3), activation=relu),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation=relu),
        MaxPooling2D(2, 2),
        # Conv2D(64, (3, 3), activation=relu),
        # MaxPooling2D(2, 2),
        # Conv2D(64, (3, 3), activation=relu),
        # MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation=relu),
        Dense(1, activation=sigmoid)
    ]
)
model.summary()

/Users/sstonn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sstonn/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     9,470,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,494,561 (36.22 MB)

 Trainable params: 9,494,561 (36.22 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    optimizer=RMSprop(
        learning_rate=0.001
    ),
    loss="binary_crossentropy",
    metrics=[
        "accuracy"
    ]
)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale=1/255).flow_from_directory(
    "./datasets/horse-or-human/",
    target_size=(150, 150),
    batch_size=128,
    class_mode="binary"
)

validation_data = ImageDataGenerator(rescale=1/255).flow_from_directory(
    "./datasets/horse-or-human-validation/",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary"
)

Found 1027 images belonging to 2 classes.
Found 1283 images belonging to 2 classes.


In [6]:
history = model.fit(
    train_data,
    steps_per_epoch=8,
    epochs=30,
    verbose=1,
    validation_data=validation_data,
    validation_steps=8
)

Epoch 1/30


/Users/sstonn/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 418ms/step - accuracy: 0.4783 - loss: 1.6701 - val_accuracy: 0.5117 - val_loss: 0.6856
Epoch 2/30
1/8 ━━━━━━━━━━━━━━━━━━━━ 2s 294ms/step - accuracy: 0.5000 - loss: 0.6867

2024-12-29 16:27:36.813621: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.5000 - loss: 0.6867 - val_accuracy: 0.6445 - val_loss: 0.6683
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 345ms/step - accuracy: 0.7031 - loss: 0.6368 - val_accuracy: 0.8906 - val_loss: 0.4666
Epoch 4/30
1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 254ms/step - accuracy: 0.8906 - loss: 0.4610

...

2024-12-29 16:28:53.761719: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.9844 - loss: 0.0362 - val_accuracy: 0.9375 - val_loss: 0.3643
Epoch 27/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 376ms/step - accuracy: 0.9882 - loss: 0.0355 - val_accuracy: 0.9688 - val_loss: 0.3435
Epoch 28/30
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.0026

2024-12-29 16:29:00.382646: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9688 - val_loss: 0.2558
Epoch 29/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 367ms/step - accuracy: 0.8422 - loss: 0.6663 - val_accuracy: 0.9648 - val_loss: 0.2372
Epoch 30/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9688 - loss: 0.0665 - val_accuracy: 1.0000 - val_loss: 0.0013


2024-12-29 16:29:07.093837: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-12-29 16:29:07.100573: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [7]:
!pip3 install pillow-avif-plugin
!pip3 install --upgrade pillow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from PIL import Image

test_dir = "./test/horse-or-human"

uploaded = listdir(test_dir)

for fn in uploaded:
  path = join(test_dir, fn)

  if (fn == ".DS_Store"):
    continue

  # predicting images
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[0.26749521]
image4.png is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[0.00133982]
image5.png is a horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[0.9999906]
image1.png is a human
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[0.9925224]
image2.png is a human
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[0.5623463]
image3.png is a human
